In [20]:
import os
import json
import asyncio
import aiohttp
import random
import pandas as pd
import base64
from datetime import datetime, timedelta
from dotenv import load_dotenv

load_dotenv()

ADO_ORG_URL = f'https://dev.azure.com/{os.getenv("ADO_ORG")}'
ADO_PAT = os.getenv("ADO_PAT")
ADO_PROJECT = os.getenv("ADO_PROJECT")

DEBUG = False

START_DATE = datetime(2024, 1, 1)  # inclusive
END_DATE = datetime(2024, 12, 31)  # exclusive
FILTER_USER = False  # set to False to get commit data for all users
USER = "example@example.com"

BASE_URL = f"https://microsoft.visualstudio.com/{ADO_PROJECT}/_apis/git/repositories"

HEADERS = {
    "Host": "microsoft.visualstudio.com",
    "User-Agent": "python/3.11.8 (Windows-10-10.0.22635-SP0) msrest/0.7.1 azure-devops/5.1.0b4 devOpsCli/1.0.1",
    "Accept-Encoding": "gzip, deflate",
    "Accept": "application/json;api-version=5.0",
    "Connection": "keep-alive",
    "Content-Type": "application/json; charset=utf-8",
    "Authorization": f'Basic {base64.b64encode(f":{ADO_PAT}".encode()).decode()}',
}


async def fetch_repositories(session: aiohttp.ClientSession) -> dict:
    """
    Fetch all repositories and return a dict mapping repo_id to repo_name.
    """
    async with session.get(BASE_URL, headers=HEADERS) as response:
        if response.status == 200:
            data = await response.json()
            repos = data.get("value", [])
            return {
                repo["id"]: repo["name"]
                for repo in repos
                if repo["isDisabled"] == False
            }
        else:
            print(f"Failed to fetch repositories. Status code: {response.status}")
            return {}


def summarize_commit(commit_json: dict) -> dict:
    """
    Given a single commit JSON structure from Azure DevOps,
    produce a summary of key information.
    """
    commit_id = commit_json.get("commitId", "")
    author_info = commit_json.get("author", {})
    author_name = author_info.get("name", "")
    author_email = author_info.get("email", "")
    commit_date = author_info.get("date", "")
    comment = commit_json.get("comment", "")

    total_changed_files = []
    changes = commit_json.get("changes", []).get("changes", [])
    for change in changes:
        if change.get("item", {}).get("gitObjectType") == "blob":
            filename = change.get("item", {}).get("path", "").split("/")[-1]
            total_changed_files.append(filename)
    summary = {
        "commitId": commit_id,
        "authorName": author_name,
        "authorEmail": author_email,
        "date": commit_date,
        "comment": comment,
        "files": total_changed_files,
        "changedFilesCount": len(total_changed_files),
    }
    return summary


async def fetch_commits_for_day(
    session: aiohttp.ClientSession,
    repo_id: str,
    repo_name: str,
    day_start: datetime,
    day_end: datetime,
    filter_user: bool = False,
) -> pd.DataFrame:
    """
    Fetch commits for a single day range [day_start, day_end) with pagination.
    Then, for each commit, fetch commit details and summarize.
    Returns a DataFrame of commit summaries.
    """

    from_str = day_start.isoformat() + "Z"
    to_str = day_end.isoformat() + "Z"

    email_filter = USER

    if filter_user:
        url = (
            f"{BASE_URL}/{repo_id}/commits?"
            f"searchCriteria.fromDate={from_str}"
            f"&searchCriteria.toDate={to_str}"
            f"&searchCriteria.$top=1000"
            f"&searchCriteria.author={email_filter}"
            f"&api-version=7.1"
        )
    else:
        url = (
            f"{BASE_URL}/{repo_id}/commits?"
            f"searchCriteria.fromDate={from_str}"
            f"&searchCriteria.toDate={to_str}"
            f"&searchCriteria.$top=1000"
            f"&api-version=7.1"
        )

    all_commits = []

    while url:
        async with session.get(url, headers=HEADERS, timeout=10000) as response:
            if response.status == 200:
                raw_text = await response.text()
                if DEBUG:
                    with open(
                        f"raw_responses/{repo_name}_{random.randint(0, 1000)}.json",
                        "w",
                        encoding="utf-8",
                    ) as f:
                        f.write(f"{response.headers}\n{raw_text}")
                data = json.loads(raw_text)

                batch_commits = data.get("value", [])
                all_commits.extend(batch_commits)
                link_header = response.headers.get("Link", "")
                next_url = None
                if link_header and 'rel="next"' in link_header:
                    parts = link_header.split(";")
                    if parts:
                        first_part = parts[0].strip()
                        if first_part.startswith("<") and first_part.endswith(">"):
                            next_url = first_part[1:-1]

                if next_url and next_url != url:
                    url = next_url
                else:
                    url = None
            else:
                print(
                    f"Failed to fetch commits for repository '{repo_name}' (ID: {repo_id}) "
                    f"on {day_start.date()}. Status code: {response.status}"
                )
                break

    commit_summaries = []
    for commit in all_commits:
        author = commit.get("author", {}).get("name", "unknown_author")
        commit_id = commit.get("commitId")
        commit_url = commit.get("url")

        if commit_url and commit_id:
            async with session.get(commit_url, headers=HEADERS) as response:
                if response.status == 200:
                    commit_data = await response.json()
                    if commit_data:
                        changes_url = (
                            commit_data.get("_links", {}).get("changes", {}).get("href")
                        )
                        if changes_url:
                            changes_url = f"{changes_url}?includeDiff=true"
                            async with session.get(
                                changes_url, headers=HEADERS
                            ) as changes_resp:
                                if changes_resp.status == 200:
                                    changes_data = await changes_resp.json()
                                    commit_data["changes"] = changes_data

                        commit_data["summary"] = summarize_commit(commit_data)
                        commit_data["summary"]["repoName"] = repo_name
                        if DEBUG:
                            with open(
                                f"commits/{author}_{commit_id}.json",
                                "w",
                                encoding="utf-8",
                            ) as f:
                                f.write(json.dumps(commit_data, indent=4))

                        commit_summaries.append(commit_data["summary"])

    if commit_summaries:
        return pd.DataFrame(commit_summaries)
    else:
        return pd.DataFrame(
            columns=["commitId", "authorName", "comment", "repoName", "date"]
        )


async def fetch_commits_for_repo(
    session: aiohttp.ClientSession,
    repo_id: str,
    repo_name: str,
    start_date: datetime,
    end_date: datetime,
) -> pd.DataFrame:
    """
    Orchestrate day-by-day async calls to fetch commits for an entire date range.
    Returns a combined DataFrame.
    """

    tasks = []
    current_date = start_date
    while current_date < end_date:
        next_date = current_date + timedelta(days=1)
        task = asyncio.create_task(
            fetch_commits_for_day(
                session,
                repo_id,
                repo_name,
                current_date,
                next_date,
                filter_user=FILTER_USER,
            )
        )
        tasks.append(task)
        current_date = next_date

    daily_dfs = await asyncio.gather(*tasks)

    if daily_dfs:
        return pd.concat(daily_dfs, ignore_index=True)
    else:
        return pd.DataFrame()

In [ ]:
async with aiohttp.ClientSession() as session:
    repo_id_name_map = await fetch_repositories(session)
    all_repo_ids = list(repo_id_name_map.keys())
    print(f"Found {len(all_repo_ids)} repositories.")

    tasks = [
        fetch_commits_for_repo(
            session, repo_id, repo_id_name_map[repo_id], START_DATE, END_DATE
        )
        for repo_id in all_repo_ids
    ]

    results = await asyncio.gather(*tasks)
    data = pd.concat(results)
    data.reset_index(drop=True, inplace=True)
    data = data.infer_objects()
    data["changedFilesCount"] = data["changedFilesCount"].astype(int)

In [ ]:
data.head(15)

In [ ]:
all_files = [file for sublist in data["files"] for file in sublist]

unique_files = set(all_files)

distinct_file_count = len(unique_files)

print("Total distinct files:", distinct_file_count)

In [ ]:
data_exploded = data.explode("files")

distinct_files_per_repo = (
    data_exploded.groupby("repoName")
    .agg(distinctFileCount=("files", "nunique"))
    .sort_values(by="distinctFileCount", ascending=False)
)

distinct_files_per_repo

In [ ]:
data.groupby("repoName").agg(
    {"commitId": "nunique", "changedFilesCount": "sum"}
).sort_values(by="commitId", ascending=False)

In [ ]:
data.groupby(["authorName"]).agg(
    {"changedFilesCount": "sum", "commitId": "nunique", "repoName": "nunique"}
).sort_values("commitId", ascending=False).head(10)